# Fuzzing with Structure-aware Fuzzers

In this notebook, we demonstrate how to use a generation fuzzer that leverages Kaitai Struct to generate valid binary files from `.ksy` specifications.

## What is Kaitai Struct?

[Kaitai Struct](https://kaitai.io/) is a declarative language used to describe the structure of binary files. It allows you to:

- Define binary formats (.gif, .png, .zip, etc.)
- Automatically generate parsers in many programming languages
- Use tools to visualize and inspect binary data

A `.ksy` file defines the **schema** of the binary file format.

In this notebook, we use `.ksy` files as blueprints and **generate valid binary files** from them using a structure-aware generation fuzzer.


In [36]:
from generation_fuzzer_main import generate_binary
import os


## 🔧 Example 1: Simple Header Format

We'll begin with a  simple .ksy format description file that describes the header of a GIF image file.

Save this into 'simple_header.ksy'

```
meta:
  id: gif
  file-extension: gif
  endian: le
seq:
  - id: header
    type: header
  - id: logical_screen
    type: logical_screen
types:
  header:
    seq:
      - id: magic
        contents: 'GIF'
      - id: version
        size: 3
  logical_screen:
    seq:
      - id: image_width
        type: u2
      - id: image_height
        type: u2
      - id: flags
        type: u1
      - id: bg_color_index
        type: u1
      - id: pixel_aspect_ratio
        type: u1
```

In [37]:
# Save the .ksy file from notebook
ksy_content = '''
meta:
  id: gif
  file-extension: gif
  endian: le
seq:
  - id: header
    type: header
  - id: logical_screen
    type: logical_screen
types:
  header:
    seq:
      - id: magic
        contents: 'GIF'
      - id: version
        type: str
        size: 3
        encoding: ASCII
  logical_screen:
    seq:
      - id: image_width
        type: u2
      - id: image_height
        type: u2
      - id: flags
        type: u1
      - id: bg_color_index
        type: u1
      - id: pixel_aspect_ratio
        type: u1
'''

with open('simple_header.ksy', 'w') as f:
    f.write(ksy_content)


## Generate Binary File from KSY

Now, we use our structure-aware generator to produce a well-formed binary file conforming to `simple_header.ksy`.


In [38]:
# Generate binary using your fuzzer
output_directory='testcases'
output_file = generate_binary('simple_header.ksy', output_directory)

# Check file size
os.path.getsize(output_file)

Dependency graph:  {'header': set(), 'logical_screen': set()}
Processing Order:  ['header', 'logical_screen']
USER DEFINED TYPE IS   header
Initial Parent string in find_userdefined data_tree
Here1
Type entry in handle_type {'seq': [{'id': 'magic', 'contents': 'GIF'}, {'id': 'version', 'type': 'str', 'size': 3, 'encoding': 'ASCII'}]}
Dependency graph:  {'version': set(), 'magic': set()}
Processing Order:  ['version', 'magic']
USER DEFINED TYPE IS   logical_screen
Initial Parent string in find_userdefined data_tree
Here1
Type entry in handle_type {'seq': [{'id': 'image_width', 'type': 'u2'}, {'id': 'image_height', 'type': 'u2'}, {'id': 'flags', 'type': 'u1'}, {'id': 'bg_color_index', 'type': 'u1'}, {'id': 'pixel_aspect_ratio', 'type': 'u1'}]}
Dependency graph:  {'image_width': set(), 'image_height': set(), 'flags': set(), 'bg_color_index': set(), 'pixel_aspect_ratio': set()}
Processing Order:  ['image_width', 'image_height', 'flags', 'bg_color_index', 'pixel_aspect_ratio']


13

In [43]:
import subprocess

print("Hexdump of the generated file:")
!hexdump -C testcases/gif.gif | head -n 20


Hexdump of the generated file:
00000000  47 49 46 5a 4c 71 0a 01  3d a7 99 7b 26           |GIFZLq..=..{&|
0000000d


## Use a Dummy Consumer Program

For illustration, let's create a dummy Python parser using the Kaitai-generated Python code.

In practice, this could be a real program that accepts binary input — an image viewer, network protocol handler, etc.


You can generate a Python parser from a .ksy file using the Kaitai Struct compiler:
```
# Install Kaitai Struct Compiler
brew install kaitai-struct-compiler       # On macOS (Homebrew)
# OR
sudo apt install kaitai-struct-compiler   # On Ubuntu/Debian

# Then compile a .ksy file to Python
kaitai-struct-compiler -t python simple_header.ksy
```
This creates a Python file like gif.py, which you can then import.

In [41]:
# Assume Kaitai Struct compiler was run earlier to produce `simple_header.py`

from gif import Gif  # auto-generated by Kaitai Struct compiler

# Read the generated binary
with open(output_file, 'rb') as f:
    parsed = Gif.from_bytes(f.read())

# Print parsed values
print("Parsed Output:")
print("Header:")
print(f"  Magic: {parsed.header.magic}")
print(f"  Version: {parsed.header.version}")

print("Logical Screen Descriptor:")
print(f"  Image Width: {parsed.logical_screen.image_width}")
print(f"  Image Height: {parsed.logical_screen.image_height}")
print(f"  Flags: {parsed.logical_screen.flags}")
print(f"  Background Color Index: {parsed.logical_screen.bg_color_index}")
print(f"  Pixel Aspect Ratio: {parsed.logical_screen.pixel_aspect_ratio}")


Parsed Output:
Header:
  Magic: b'GIF'
  Version: ZLq
Logical Screen Descriptor:
  Image Width: 266
  Image Height: 42813
  Flags: 153
  Background Color Index: 123
  Pixel Aspect Ratio: 38


We successfully:

- Defined a simple binary format using Kaitai Struct
- Used a structure-aware generator to produce valid binary data
- Parsed the data using an auto-generated Kaitai parser

This shows that the generated files conform to the expected structure.


## Using Our Generator with PNG format

Now that we've seen how our generation fuzzer works on a simple header format, let’s demonstrate it on a more complex, real-world format — the PNG image file format, described by png.ksy.

1. Generate a PNG binary file using our fuzzer

In [54]:
# Generate binary using your fuzzer
output_directory='testcases'
png_file = generate_binary('ksy_files/png.ksy', output_directory)

# Check file size
os.path.getsize(png_file)

Dependency graph:  {'ihdr_crc': set(), 'magic': set(), 'ihdr_len': set(), 'ihdr_type': set(), 'ihdr': set(), 'chunks': set()}
Processing Order:  ['ihdr_crc', 'magic', 'ihdr_len', 'ihdr_type', 'ihdr', 'chunks']
VALUE is 13
Converting value: 13, type_field: u4, endianness: >
USER DEFINED TYPE IS   ihdr_chunk
Initial Parent string in find_userdefined data_tree
Here1
Type entry in handle_type {'doc-ref': 'https://www.w3.org/TR/PNG/#11IHDR', 'seq': [{'id': 'width', 'type': 'u4'}, {'id': 'height', 'type': 'u4'}, {'id': 'bit_depth', 'type': 'u1'}, {'id': 'color_type', 'type': 'u1', 'enum': 'color_type'}, {'id': 'compression_method', 'type': 'u1'}, {'id': 'filter_method', 'type': 'u1'}, {'id': 'interlace_method', 'type': 'u1'}]}
Dependency graph:  {'width': set(), 'height': set(), 'bit_depth': set(), 'color_type': set(), 'compression_method': set(), 'filter_method': set(), 'interlace_method': set()}
Processing Order:  ['width', 'height', 'bit_depth', 'color_type', 'compression_method', 'filter

1427

# Validating Generated PNG Files